#Assignment 1 (Optional)
In this notebook is contained the solution of the optional assignment (third point)

In [2]:
import nltk
nltk.download('dependency_treebank')

[nltk_data] Downloading package dependency_treebank to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping corpora/dependency_treebank.zip.


True

#Modifing the classifier

To modify the classifier first we extend the TransitionParser class, which is contained in the nltk.parse libraries.

Then we modify the classifier defined in the train function. Instead of an SVM classifier, an MLP one can be used insted

In [35]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression

import tempfile
import pickle

from os import remove
from copy import deepcopy
from operator import itemgetter
from sklearn.datasets import load_svmlight_file

class MyTransitionParser(TransitionParser):
  def train(self, depgraphs, modelfile, verbose=True):
          try:
              input_file = tempfile.NamedTemporaryFile(
                  prefix="transition_parse.train", dir=tempfile.gettempdir(), delete=False
              )

              if self._algorithm == self.ARC_STANDARD:
                  self._create_training_examples_arc_std(depgraphs, input_file)
              else:
                  self._create_training_examples_arc_eager(depgraphs, input_file)

              input_file.close()
              x_train, y_train = load_svmlight_file(input_file.name)
              model = MultinomialNB()
              #model = MLPClassifier(random_state=1, max_iter=300)
              #model = LogisticRegression(n_jobs=1, C=1e5)

              model.fit(x_train, y_train)
              pickle.dump(model, open(modelfile, "wb"))
          finally:
              remove(input_file.name)

In [36]:
from nltk.parse.dependencygraph import DependencyGraph
from nltk.parse import ProbabilisticProjectiveDependencyParser
from nltk.parse.transitionparser import TransitionParser
from nltk.corpus import dependency_treebank
from nltk.parse import DependencyEvaluator

print('---------- NLTK TRANSITION PARSER WITH SVM CLASSIFIER ----------')

tp = TransitionParser('arc-standard')
tp.train(dependency_treebank.parsed_sents()[:200], 'tp.model')
parses = tp.parse(dependency_treebank.parsed_sents()[-10:], 'tp.model')

de = DependencyEvaluator(parses, dependency_treebank.parsed_sents()[-10:])
las, uas = de.eval()

print('Evaluation: ')
print('\tlabeled attachments:', las)
print('\tunlabeled attachments:', uas)

print('\n---------- NLTK TRANSITION PARSER WITH MLP CLASSIFIER ----------')

tp = MyTransitionParser('arc-standard')
tp.train(dependency_treebank.parsed_sents()[:200], 'tp.model')
parses = tp.parse(dependency_treebank.parsed_sents()[-10:], 'tp.model')

de = DependencyEvaluator(parses, dependency_treebank.parsed_sents()[-10:])
las, uas = de.eval()

print('Evaluation: ')
print('\tlabeled attachments:', las)
print('\tunlabeled attachments:', uas)

---------- NLTK TRANSITION PARSER WITH SVM CLASSIFIER ----------
 Number of training examples : 200
 Number of valid (projective) examples : 200
[LibSVM]Evaluation: 
	labeled attachments: 0.7958333333333333
	unlabeled attachments: 0.7958333333333333

---------- NLTK TRANSITION PARSER WITH MLP CLASSIFIER ----------
 Number of training examples : 200
 Number of valid (projective) examples : 200
Evaluation: 
	labeled attachments: 0.6166666666666667
	unlabeled attachments: 0.6166666666666667


#SVM Classifier

```
labeled attachments: 0.7958333333333333
unlabeled attachments: 0.7958333333333333
```
#Linear Regression


```
labeled attachments: 0.7375
unlabeled attachments: 0.7375
  ```
#Multi-layer Perceptron Classifier

```
labeled attachments: 0.7333333333333333
unlabeled attachments: 0.7333333333333333
```



#Naive Bayes Classifier for Multinomial Model
```
labeled attachments: 0.6166666666666667
unlabeled attachments: 0.6166666666666667
```

So it seems that Linear Regression, together with MLP, are the classifiers that are closer to the SVM classifier performances. 
Even though SVM is still the best one.
